In [159]:

'''資料集來源'''
# 台股上市櫃名單一覽
# https://isin.twse.com.tw/isin/C_public.jsp?strMode=2

'''載入工具包'''
import requests as req # 拿網頁資訊用 
from bs4 import BeautifulSoup as bs # 用來解析html
from functools import cache
import json, os
from pprint import pprint # 讓載入回來的html格式好看一點

# 隨機取得 user-agent
from fake_useragent import UserAgent
ua = UserAgent(cache=True)

# 建立儲存爬蟲資訊的資料夾，不存在就新增
folderPath = 'stocks_list_TW'
if not os.path.exists(folderPath):
    os.makedirs(folderPath)

''' 使用 get 的方式下載網頁資源'''
cust_headers = {
    'user-agent' : ua.random
}

url = 'https://isin.twse.com.tw/isin/C_public.jsp?strMode=2'
res = req.get(url, headers = cust_headers)

# pprint(res.text)




In [161]:
import pandas as pd
import re

res.encoding = "big5"
df = pd.read_html(res.text)[0]
df


,0,1,2,3,4,5,6
0,有價證券代號及名稱,國際證券辨識號碼(ISIN Code),上市日,市場別,產業別,CFICode,備註
1,股票,股票,股票,股票,股票,股票,股票
2,1101 台泥,TW0001101004,1962/02/09,上市,水泥工業,ESVUFR,NaN
3,1102 亞泥,TW0001102002,1962/06/08,上市,水泥工業,ESVUFR,NaN
4,1103 嘉泥,TW0001103000,1969/11/14,上市,水泥工業,ESVUFR,NaN
...,...,...,...,...,...,...,...
26674,01003T 兆豐新光R1,TW00001003T4,2005/12/26,上市,NaN,CBCIXU,NaN
26675,01004T 土銀富邦R2,TW00001004T2,2006/04/13,上市,NaN,CBCIXU,NaN
26676,01007T 兆豐國泰R2,TW00001007T5,2006/10/13,上市,NaN,CBCIXU,NaN
26677,01009T 王道圓滿R1,TW00001009T1,2018/06/21,上市,NaN,CBCIXU,NaN


In [162]:
# 以第0行作為column名稱
df.columns = df.iloc[0]
# 刪除第 0, 1 rows
df = df.iloc[2:]

# 刪除 row and column 的 NaN 



In [166]:
df

,有價證券代號及名稱,國際證券辨識號碼(ISIN Code),上市日,市場別,產業別,CFICode
2,1101 台泥,TW0001101004,1962/02/09,上市,水泥工業,ESVUFR
3,1102 亞泥,TW0001102002,1962/06/08,上市,水泥工業,ESVUFR
4,1103 嘉泥,TW0001103000,1969/11/14,上市,水泥工業,ESVUFR
5,1104 環泥,TW0001104008,1971/02/01,上市,水泥工業,ESVUFR
6,1108 幸福,TW0001108009,1990/06/06,上市,水泥工業,ESVUFR
...,...,...,...,...,...,...
26674,01003T 兆豐新光R1,TW00001003T4,2005/12/26,上市,NaN,CBCIXU
26675,01004T 土銀富邦R2,TW00001004T2,2006/04/13,上市,NaN,CBCIXU
26676,01007T 兆豐國泰R2,TW00001007T5,2006/10/13,上市,NaN,CBCIXU
26677,01009T 王道圓滿R1,TW00001009T1,2018/06/21,上市,NaN,CBCIXU


In [167]:
len(df.columns)
df['產業別'].notnull()
# df

2         True
3         True
4         True
5         True
6         True
         ...  
26674    False
26675    False
26676    False
26677    False
26678    False
Name: 產業別, Length: 26677, dtype: bool

In [165]:
# df = df[df['備註'].notna()]
# df[df['產業別'].notna()]

# 刪除 last columns
df = df.iloc[:,:len(df.columns)-1]
# 刪除 row 方向有 NaN 欄位
# df = df.dropna('')





In [168]:
df = df[~df.有價證券代號及名稱.str.contains('[^0-9]{4}')]

In [169]:
df = df.set_index('有價證券代號及名稱')
df

,國際證券辨識號碼(ISIN Code),上市日,市場別,產業別,CFICode
有價證券代號及名稱,,,,,
1101 台泥,TW0001101004,1962/02/09,上市,水泥工業,ESVUFR
1102 亞泥,TW0001102002,1962/06/08,上市,水泥工業,ESVUFR
1103 嘉泥,TW0001103000,1969/11/14,上市,水泥工業,ESVUFR
1104 環泥,TW0001104008,1971/02/01,上市,水泥工業,ESVUFR
1108 幸福,TW0001108009,1990/06/06,上市,水泥工業,ESVUFR
...,...,...,...,...,...
ETN,ETN,ETN,ETN,ETN,ETN
特別股,特別股,特別股,特別股,特別股,特別股
普通股,普通股,普通股,普通股,普通股,普通股


In [172]:
df.to_csv(path_or_buf='./stocks_list_TW/stocks.csv', encoding='big5')